In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain_google_vertexai import ChatVertexAI

In [6]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [14]:
from langchain_community.document_loaders import WebBaseLoader
import bs4


In [60]:
from PyPDF2 import PdfReader
folder_path = 'data'
text=""
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(folder_path, filename)
        print(pdf_path)
        pdf_reader= PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text+= page.extract_text()

data\Arbitration and Conciliation Act, 1996.pdf
data\Consumer Protection Act, 2019.pdf
data\Hindu Succession Act, 1956.pdf
data\Indian Succession Act, 1925.pdf
data\Minimum Wages Act, 1948.pdf
data\Specific Relief Act, 1963.pdf
data\The Copyright Act, 1957.pdf
data\THE DELHI RENT CONTROL ACT, 1958.pdf
data\THE HINDU MARRIAGE ACT, 1955.pdf
data\THE INDIAN CONTRACT ACT, 1872.pdf
data\THE INDUSTRIAL DISPUTES ACT, 1947.pdf
data\The Limitation Act, 1963.pdf
data\THE SPECIAL MARRIAGE ACT, 1954.pdf
data\THE TRANSFER OF PROPERTY ACT, 1882.pdf


In [61]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [62]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
splits = text_splitter.split_text(text)
splits

['1 \n THE ARBITRATION AND CONCILIATION ACT, 1996  \n_______  \nARRA NGEMENT OF SECTIONS  \n \n_______  \nPREAMBLE  \nPRELIMINARY  \nSECTIONS . \n1. Short title, extent and commencement.  \nPART I  \nARBITRA TION  \nCHAPTER I  \nGeneral provisions  \n2. Definitions.  \n3. Receipt of written communications.  \n4. Waiver of right to  object.  \n5. Extent of judicial intervention.  \n6. Administrative assistance.  \nCHAPTER II  \nArbitration agreement  \n7. Arbitration agreement.  \n8. Power to refer parties to arbitration where there is an arbitration agreement.  \n9. Interim mea sures, etc., by Court.  \nCHAPTER  III \nComposition of arbitral tribunal  \n10. Number of arbitrators.  \n11. Appointment of arbitrators.  \n11A. Power of Central Government to amend Fourth Schedule . \n12. Grounds for challenge.  \n13. Challenge procedure.  \n14. Failure or impossibility to act.  \n15. Termination of mandate and subs titution of arbitrator.  \nCHAPTER IV  \nJurisdiction of arbitral tribunals  

In [63]:

vectorstore = FAISS.from_texts(splits, embedding=GoogleGenerativeAIEmbeddings(model = "models/embedding-001"))
retriever = vectorstore.as_retriever()

In [85]:
vectorstore.save_local("civilLaw_database")

In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [77]:
system_prompt = (
    "You are an assistant specialized in Indian civil law. "
    "Answer legal queries based on the provided context. "
    "If you don't know the answer, say 'I don't know'. "
    "Provide a concise answer. List down the references with REFERENCES - format at the end of each response"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [78]:
response = rag_chain.invoke({"input": "What are my rights as a tenant residing in Delhi under Indian law?"})
response["answer"]

'As a tenant residing in Delhi, you are protected under the Delhi Rent Control Act, 1958. Here are some of your key rights:\n\n* **Protection against illegal rent increases:** Your landlord cannot charge you rent exceeding the "standard rent" as determined under the Act.\n* **Protection against illegal eviction:**  Your landlord cannot evict you without a valid reason and following due process. \n* **Right to deposit rent with the Controller:** If your landlord refuses to accept rent or there is a dispute about rent payment, you can deposit the rent with the Rent Controller.\n* **Right to repair:** Your landlord is obligated to keep the premises in good repair.\n* **Right to notice before eviction:** Your landlord must give you proper notice before attempting to evict you, and you have the right to contest the eviction in a court of law.\n* **Right to sublet:** You can sublet your premises if you obtain written consent from your landlord.\n\n**Important Considerations:**\n\n* **Specifi

In [79]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [80]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [81]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [82]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [86]:
conversational_rag_chain.invoke(
    {"input": "What are my rights as a tenant residing in Delhi under Indian law?"},
    config={
        "configurable": {"session_id": "abc123"}
    }, 
)["answer"]

'As a tenant residing in Delhi, you have several rights protected under the Delhi Rent Control Act, 1958. These include:\n\n**Protection from Unlawful Eviction:**\n\n* **Valid Reason:** You cannot be evicted without a valid reason listed in the Act, such as non-payment of rent, misuse of the premises, or the landlord\'s genuine need for the property. \n* **Legal Proceedings:**  Your landlord must follow proper legal procedures, including giving you notice and allowing you to be heard before the Rent Controller.\n\n**Protection from Unreasonable Rent Increases:**\n\n* **Standard Rent:**  The Rent Controller determines a "standard rent" based on factors like the market value, age, and condition of the property. Your rent cannot be increased beyond this standard rent.\n\n**Right to a Fair Rent:**\n\n* **Rent Controller\'s Role:** The Rent Controller can fix a "standard rent" if there is a dispute between you and your landlord. \n\n**Right to Repair:**\n\n* **Landlord\'s Duty:** Your landl

In [87]:
conversational_rag_chain.invoke(
    {"input": "What did I ask before"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'You asked: "What are my rights as a tenant residing in Delhi under Indian law?" \n'